# Overtaking control using kinematic bicycle model
In this task, you will design a nonlinear MPC on a kinematic bicycle car model to overtake the leading vehicle.

You will need to fill the **student_controller()** function and return the required utils for *Casadi* nonlinear programming solver. 

## System model

The leading vehicle is cruising with velocity $\bar{v}_x$ along the center of a straight lane. Let the left hand side of the driving direction be the positive direction of $y$ and the $y$ position of leading vehicle as $0$.

The state of the ego vehicle is defined as the position with respect to the leading car $\left[x, y\right]$, yaw angle $\psi$, and the velocity $v$. The vehicle is controlled by the steering angle of front wheel $\delta$ and the acceleration $a$.

Thus, the dynamics of the vehicle can be expressed as:

$$
\frac{d}{dt} \begin{bmatrix} x \\ y \\ \psi \\ v \end{bmatrix}
= \begin{bmatrix} v\cos(\psi + \beta) - \bar{v}_x \\
                  v\sin(\psi + \beta)\\
                  \frac{v}{L_r} \sin\beta \\
                  a
  \end{bmatrix}, \text{with } \beta := \arctan(\frac{L_r}{L_r + L_f} \arctan \delta).
$$

where $L_r$ and $L_f$ are distances from the rear or front axes to the center of the vehicle. 

We apply the explicit Euler integration to obtain the discrete-time dynamics model of $\dot{s}=f(s, u) \ \ \text{as}\ \ s_{k+1} = s_{k} + f(s_k, u_k) \Delta t.$

In this task, we assume $\Delta t = 0.1s$ in both simulation and controller design.


<div>
<img src="takeover.png" width="800" align="left"/>
</div>

## Controller design

You are supposed to design a MPC controller of the following form to takeover the leading vehicle with desired velocity and go back to the same lane as the leading vehicle.

$
\begin{equation}
    \begin{aligned}
    \min_{s_k, u_k} & \ C_{N} + \sum_{k=0}^{N-1} C_{k} \\ 
    \text{s.t. } & s_{k+1} = s_{k} + f(s_k, u_k) \Delta t, \ k = 0, \dots, N-1, \\
                 & Cons1_k, Cons2_k, Cons3_k, Cons4_k, \\
                 & s_0 = s_{init}.
    \end{aligned}
\end{equation}
$

Here, $C_{N}$ is the terminal cost function, $C_{k}$ is the running cost function.

### Cost function

Consider these goals when designing your cost function:

C1: Track a desired longitudinal velocity. 

C2: Regularize the lateral velocity and yaw rate.

C3: Encourage the car to stay at the same lane as the leading vehicle. 

C4: Regularize the control inputs. 

Hint: 
1. When the car successfully takes over the car, the car will return to the original lane. Thus, enforce the position constraints (C3) and the car will automatically have zero lateral and angular velocity (C2).
2. It is advised to implement (C1) and (C4) first, tune the coefficients so that all the constraint are met, then add (C3).
3. The terminal cost function $C_{N}$ should not include (C4) and could be larger to ensure the goal is achieved.

### Constraints

#### Cons1: Collision avoidance:
We consider a elliposoidal safety set for the vehicle when overtaking the car

$(\frac{x_k}{r_x})^2 + (\frac{y_k}{r_y})^2 - 1 \ge 0 $, 

We assign $r_x = 30 m, r_y = 2 m$.

#### Cons1: Maximum lateral acceleration
We restrict the lateral acceleration $a_y$ by:
$|a_y| \le 0.5 \mu g $, where $\mu$ is the coefficient of friction and $g$ is the gravity. 

Hint: The lateral acceleration $ a_y = v \dot{\psi}$. What is $\dot{\psi}?$

We assume the $\mu = 0.6$ and $g = 9.81 m / s$.

#### Cons2: lane keeping
We restrict the car to only overtake the car using the left lane and do not cross the right lane.

$y_R \le y \le y_L$

We assume $y_L = 3m$ and $y_R = -1m$.

#### Cons3: steering rate ####
The steering angle of the vehicle can not change too fast:

<!-- $|\delta_{k+1} - \delta_{k}| \le |\dot{\delta}|_{\max} \Delta t $ -->
$\|\dot{\delta}\| \le |\dot{\delta}|_{max} = 0.6 \text{rad}/s$. 

#### Cons4: input bounds
The input is within the bounds:

$-10 m/s^2 = a_{\min} \le a \le a_{\max} = 4 m / s^2$

$-0.6rad = \delta_{\min} \le \delta \le \delta_{\max} = 0.6rad $

## Submission detail

### Controller input
The autograder will provide the initial condition of the state, leader car velocity, desired longitudinal speed and the steering angle from the last step. 

### Controller output
You are supposed to fill the **nmpc_controller()** in **nmpc_controller.py**, which will return an instance of the nonlinear programming problem, the planning horizon, shapes of the varaibles, and bounds. 

In [1]:
%load_ext autoreload
%autoreload 2
from utils import eval_controller, plot_results

In [ ]:
# Initial test case
x0 = [
    -49., # x: x distance between our car and leader car
    0.0,  # y: y distance between our car and leader car
    0.0,  # yaw: yaw angle of our car
    50.,  # v: velocity of our car
    20.,  # v_x_leader: x velocity of leader car
    0.0,  # v_y_leader: y velocity of leader car
    50.0  # v_x_desired: desired takeover x velocity of our car
]
x0 = [-50., -0.5, -0.06, 30., 30., 0., 40.]
xt, ut = eval_controller(x0)
plot_results(x0, xt, ut)
